In [ ]:
print("GPU IS READY")

In [1]:
import pandas as pd

# Paths
local_csv = r"/kaggle/input/apparel-dataset-test/NEW_Main.csv"  # Read-only CSV
kaggle_csv = r"/kaggle/working/New_Main.csv"  # New CSV for Kaggle

# Load the CSV in read-only mode
df = pd.read_csv(local_csv)

# Path correction logic
kaggle_base_path = "/kaggle/input/apparel-dataset-test/ALL IMAGES/ALL IMAGES"

# Update paths
df["image path"] = df.apply(
    lambda row: f"{kaggle_base_path}/{row['name']}/{row['image name']}", axis=1
)

# Save the updated CSV for Kaggle
df.to_csv(kaggle_csv, index=False)

print(f"✅ CSV saved successfully at: {kaggle_csv}")
print(df.head())

✅ CSV saved successfully at: /kaggle/working/New_Main.csv
                           name                            image name  \
0  black_checkered_casual_shirt  black_checkered_casual_shirt_100.jpg   
1  black_checkered_casual_shirt  black_checkered_casual_shirt_102.jpg   
2  black_checkered_casual_shirt  black_checkered_casual_shirt_104.jpg   
3  black_checkered_casual_shirt  black_checkered_casual_shirt_106.jpg   
4  black_checkered_casual_shirt  black_checkered_casual_shirt_108.jpg   

                                          image path  \
0  /kaggle/input/apparel-dataset-test/ALL IMAGES/...   
1  /kaggle/input/apparel-dataset-test/ALL IMAGES/...   
2  /kaggle/input/apparel-dataset-test/ALL IMAGES/...   
3  /kaggle/input/apparel-dataset-test/ALL IMAGES/...   
4  /kaggle/input/apparel-dataset-test/ALL IMAGES/...   

                     attributes  
0  black,checkered,casual,shirt  
1  black,checkered,casual,shirt  
2  black,checkered,casual,shirt  
3  black,checkered,casual,shir

In [2]:
df.head()

,name,image name,image path,attributes
0,black_checkered_casual_shirt,black_checkered_casual_shirt_100.jpg,/kaggle/input/apparel-dataset-test/ALL IMAGES/...,"black,checkered,casual,shirt"
1,black_checkered_casual_shirt,black_checkered_casual_shirt_102.jpg,/kaggle/input/apparel-dataset-test/ALL IMAGES/...,"black,checkered,casual,shirt"
2,black_checkered_casual_shirt,black_checkered_casual_shirt_104.jpg,/kaggle/input/apparel-dataset-test/ALL IMAGES/...,"black,checkered,casual,shirt"
3,black_checkered_casual_shirt,black_checkered_casual_shirt_106.jpg,/kaggle/input/apparel-dataset-test/ALL IMAGES/...,"black,checkered,casual,shirt"
4,black_checkered_casual_shirt,black_checkered_casual_shirt_108.jpg,/kaggle/input/apparel-dataset-test/ALL IMAGES/...,"black,checkered,casual,shirt"


In [3]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
csv_path = r"/kaggle/working/New_Main.csv"
root_dir = r"/kaggle/input/apparel-dataset-test/ALL IMAGES/ALL IMAGES"

In [5]:
data = pd.read_csv(csv_path, on_bad_lines='skip')
data = data.dropna().reset_index(drop=True)

In [6]:
colors = ['white', 'blue', 'black', 'red', 'green','brown']
types = ["t-shirt", "shirt", "polo", "formal shirt", "casual shirt","dress","pants","shoes","shorts"]
patterns = ["plain", "checkered", "striped", "floral", "dotted", "printed"]
classes = colors + types + patterns
print("Total classes:", len(classes))

Total classes: 21


In [7]:
# Function to encode labels from CSV
def encode_label(label, classes_list=classes):
    target = torch.zeros(len(classes_list))
    for l in label.split(","):  # Ensure proper splitting by comma
        l = l.strip()  # Remove any leading/trailing spaces
        if l in classes_list:
            idx = classes_list.index(l)
            target[idx] = 1
    return target

# Apply encoding to the dataset
data["encoded_target"] = data["attributes"].apply(encode_label)

# Decode labels for verification
def decode_target(target, threshold=0.4):
    result = []
    for i, x in enumerate(target):
        if x >= threshold:
            result.append(classes[i])
    return result

# Verify the encoding
print(data[["attributes", "encoded_target"]])

                         attributes  \
0      black,checkered,casual,shirt   
1      black,checkered,casual,shirt   
2      black,checkered,casual,shirt   
3      black,checkered,casual,shirt   
4      black,checkered,casual,shirt   
...                             ...   
37713         white,striped,t-shirt   
37714         white,striped,t-shirt   
37715         white,striped,t-shirt   
37716         white,striped,t-shirt   
37717         white,striped,t-shirt   

                                          encoded_target  
0      [tensor(0.), tensor(0.), tensor(1.), tensor(0....  
1      [tensor(0.), tensor(0.), tensor(1.), tensor(0....  
2      [tensor(0.), tensor(0.), tensor(1.), tensor(0....  
3      [tensor(0.), tensor(0.), tensor(1.), tensor(0....  
4      [tensor(0.), tensor(0.), tensor(1.), tensor(0....  
...                                                  ...  
37713  [tensor(1.), tensor(0.), tensor(0.), tensor(0....  
37714  [tensor(1.), tensor(0.), tensor(0.), tensor(0....  


In [8]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform = T.Compose([
    T.Resize((128, 128)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(2),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])

In [9]:
class MyDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['image path'].strip()
        img_path = os.path.join(self.root_dir, img_name)

        # Check if image exists
        if not os.path.exists(img_path):
            print(f"Missing image: {img_path}")
            return torch.zeros(3, 128, 128), torch.zeros(len(classes))  # Dummy data

        try:
            img = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return torch.zeros(3, 128, 128), torch.zeros(len(classes))  # Return dummy data

        if self.transform:
            img = self.transform(img)

        # Get label from CSV
        label = self.data.iloc[idx]["encoded_target"]

        # Ensure label is a tensor
        if isinstance(label, str):
            label = torch.tensor(eval(label), dtype=torch.float32)  # Convert stringified tensor to tensor
        else:
            label = torch.tensor(label, dtype=torch.float32)

        return img, label

In [10]:
dataset = MyDataset(data, root_dir, transform=transform)
train_ds, val_ds = train_test_split(dataset, test_size=0.15, random_state=42)

In [11]:
batch_size = 32
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size * 2)

In [12]:
def F_score(output, label, threshold=0.7, beta=1):  
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

# Model Training Base Class
class MultilabelImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, targets = batch 
        out = self(images)                            
        loss = F.binary_cross_entropy(out, targets)   
        return loss    

    def validation_step(self, batch):
        images, targets = batch 
        out = self(images)                           
        loss = F.binary_cross_entropy(out, targets)  
        score = F_score(out, targets)                
        return {'val_loss': loss.detach(), 'val_score': score.detach()}      

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       
        batch_scores = [x['val_score'] for x in outputs]    
        epoch_score = torch.stack(batch_scores).mean()      
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}    

    def epoch_end(self, epoch, result):                     
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))

In [13]:
class ResNet15(MultilabelImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()     
        # Input 3 x 128 x 128
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.res1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.MaxPool2d(4),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.res2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 512, kernel_size=3, padding=1),
            nn.MaxPool2d(4),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        self.res3 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.MaxPool2d(4),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True)
        )
        self.res4 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True)
        )
        self.classifier = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(1024 * 1 * 1, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.res1(out) + out
        out = self.conv2(out)
        out = self.res2(out) + out
        out = self.conv3(out)
        out = self.res3(out) + out
        out = self.conv4(out)
        out = self.res4(out) + out
        out = self.classifier(out)
        out = torch.sigmoid(out)  # Use sigmoid for multi-label classification
        return out

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet15(3, len(classes)).to(device)
print(device)

cuda


In [15]:
def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            images, targets = batch
            images, targets = images.to(device), targets.to(device)  # Move data to device
            loss = model.training_step((images, targets))
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

# Define the get_lr function (used in fit_one_cycle)
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [16]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = []
    for batch in val_loader:
        images, targets = batch
        images, targets = images.to(device), targets.to(device)  # Move data to device
        outputs.append(model.validation_step((images, targets)))
    return model.validation_epoch_end(outputs)

In [17]:
epochs = 50
max_lr = 0.001
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [18]:
history = fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

  0%|          | 0/1002 [00:00<?, ?it/s]

Epoch [0], last_lr: 0.0001, train_loss: 0.2412, val_loss: 0.1977, val_score: 0.5677


  0%|          | 0/1002 [00:00<?, ?it/s]

Epoch [1], last_lr: 0.0001, train_loss: 0.1867, val_loss: 0.1775, val_score: 0.6372


  0%|          | 0/1002 [00:00<?, ?it/s]

...

Epoch [47], last_lr: 0.0000, train_loss: 0.0039, val_loss: 0.2776, val_score: 0.7692


  0%|          | 0/1002 [00:00<?, ?it/s]

Epoch [48], last_lr: 0.0000, train_loss: 0.0038, val_loss: 0.2773, val_score: 0.7706


  0%|          | 0/1002 [00:00<?, ?it/s]

Epoch [49], last_lr: 0.0000, train_loss: 0.0037, val_loss: 0.2822, val_score: 0.7697


In [22]:
def predict_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = transform(img).unsqueeze(0)  # Apply transformations and add batch dimension
    img = img.to(device)  # Move image to device
    output = model(img)
    predicted_attributes = decode_target(output[0].cpu())  # Decode the output to get the predicted labels
    return predicted_attributes

# Example usage
image_path = r"/kaggle/input/apparel-dataset-test/ALL IMAGES/ALL IMAGES/black_shorts/005b19f2af0962b0a70455fbdc269a073ca3b0bf.jpg"
print("Predicted Attributes:", predict_image(image_path))

Predicted Attributes: ['black', 'shorts']


In [23]:
torch.save(model.state_dict(), "SixthModel.pth")